![alt text](https://www.gov.br/mdh/pt-br/ondh/centrais-de-conteudo/imagens/ligue-180.png "Imagem Canal 180")


# Limpeza de Dados - Central de Atendimento à Mulher (180) 


## Sobre o Ligue 180

O Ligue 180 é um canal de atendimento brasileiro que recebe denúncias de violência contra a mulher em todo o território nacional. Ao discar 180, você pode reportar um caso de violência para que este seja encaminhado aos órgãos competentes, além de obter o apoio e as orientações necessárias. Você também pode pedir acolhimento para si ou para alguém que conheça em situação de vulnerabilidade, e pedir informações gerais sobre os direitos da mulher presentes na legislação.

Para mais informações sobre o Canal 180, acesse: 

<https://www.gov.br/mdh/pt-br/ligue180>

Os dados que utilizaremos neste projeto correspondem aos registros do Ligue 180 durante o primeiro semestre de 2020. Todas as informações aqui presentes são disponibilizadas pelo Ministério da Mulher, da Família e dos Direitos Humanos. As tabelas não contém informações pessoais das vítimas e suspeitos. Para acessar e baixar estes dados, [acesse aqui](https://www.gov.br/mdh/pt-br/acesso-a-informacao/dados-abertos/ligue180).


## Objetivos


Neste projeto, limparemos os dados do Ligue 180 para que no futuro possamos responder algumas questões relacionadas à violência contra a mulher no Brasil:

1. Como é o mapa da violência contra a mulher no Brasil atualmente?

2. Quais são os tipos de crimes mais comuns reportados através do Canal 180?

3. Como foi o crescimento dos crimes contra a mulher nos últimos anos?

4. Qual é o perfil geral das vítimas reportados no Canal 180?

As respostas para estas perguntas serão apresentadas em um Dashboard interativo, juntamente com dados de outros anos e semestres.

O passo a passo da Limpeza de Dados do Canal 180 será explicado neste Jupyter Notebook.

## Limpeza dos Dados

#### Bibliotecas e funções usadas no projeto

As linhas de código a seguir carregam as bibliotecas do Python necessárias para este projeto, a saber: Numpy, Pandas e Regular Expressions. 

In [ ]:
import numpy as np
import pandas as pd
import re

A função a seguir será utilizada para converter algumas strings de alta complexidade presentes em nosso DataFrame, gerando strings mais legíveis. Desta forma, a categorização e análise dos dados pode se tornar mais simples. A utilidade desta função ficará mais clara em pontos futuros desta análise.

In [ ]:
def CleanRowData(column):
    
    if isinstance(df_180[column][0], str):
        
        print('Convertendo strings em listas ...')
        
        # substituir caracteres ";" e "." por "__"
        # criar uma lista de strings -> a separação entre elementos ocorre onde há "__"
        df_180[column] = df_180[column].apply(lambda x: x.replace(';','__').replace('.','__').split('__'))
        
        # seleção dos índices ímpares da lista
        df_180[column] = df_180[column].apply(lambda x: x[1::2])

    print('Limpando listas ...')
    
    # listas com entradas não nulas
    list_items = df_180[column].notnull()

    all_items = set()

    for i, item in enumerate(list_items):

        if item == True:

            entry = df_180[column][i]

            for j in entry:

                # adicionar diferentes itens da coluna em um set
                all_items.add(j)
                
    return all_items

#### Abrindo e analisando o nosso arquivo .csv

O comando a seguir realizará o download dos dados referentes ao projeto. Como a planilha é pesada (possui cerca de 146 MB), este processo pode demorar alguns minutos, dependendo da velocidade de sua conexão de Internet. Alternativamente, você pode realizar o download diretamente através do portal do governo e carregar neste Jupyter Notebook especificando o diretório onde ele está contido.

In [ ]:
!wget 'https://dadosabertos.mdh.gov.br/primeiro-semestre-2020.csv'

In [ ]:
df_180 = pd.read_csv('primeiro-semestre-2020.csv')

In [ ]:
# 5 primeiras linhas do DataFrame
df_180.head()

In [ ]:
# tamanho da tabela -> (num linhas, num colunas)
df_180.shape

Podemos ver acima que a tabela com a qual estamos trabalhando possui 33 colunas. Podemos utilizar o método a seguir para investigar o nome de todas as colunas:

In [ ]:
df_180.columns

#### Limpeza dos dados

Um ponto importante em Limpeza de Dados (Data Cleaning) consiste em renomearmos as colunas de nosso DataFrame de forma que os dados nele contidos sejam facilmente acessíveis. Boas práticas incluem atribuir nomes intuitivos às colunas, evitando caracteres de espaço. No caso de nossa análise, podemos renomear as colunas da seguinte forma:

In [ ]:
colunas = ['data_denuncia','denunciante','rel_denunciante_vitima',
           'canal_atendimento','pais_vitima','uf','estado',
           'municipio','local_violacao','grupo_vitima',
           'sexo_vitima','idade_vitima','escolaridade_vitima', 
           'renda_vitima','raca_cor_vitima','deficiencia_vitima',
           'sexo_suspeito', 'idade_suspeito', 'escolaridade_suspeito',
           'renda_suspeito','raca_cor_suspeito',
           'deficiencia_suspeito','rel_suspeito_vitima',
           'denuncia_emergencial','nacionalidade_vitima',
           'nacionalidade_suspeito','pais_suspeito',
           'etnia_vitima','etnia_suspeito','hash','violacoes',
           'motivacoes','agravantes']

# atribuir nomes desejados às colunas do dataframe:
df_180.columns = colunas

Agora, sabendo o tipo de informações que estão contidas no DataFrame, podemos remover os dados redundantes ou desnecessários para nossa análise. Tendo em conta as perguntas iniciais que desejamos responder neste projeto (veja novamente em [objetivos](#Objetivos)), podemos remover as colunas referentes aos perfis dos suspeitos, com exceção das colunas que contêm o sexo dos suspeitos e a relação entre o suspeito e a vítima.

In [ ]:
df_180.drop(['idade_suspeito', 'escolaridade_suspeito', 'renda_suspeito', 'raca_cor_suspeito', 
             'deficiencia_suspeito', 'nacionalidade_suspeito', 'pais_suspeito', 'etnia_suspeito'], 
             axis=1, inplace=True)

Além das colunas que foram removidas acima, podemos remover a coluna correspondente às Unidades Federais (UF), já que há uma coluna correspondente ao Estado onde o crime ocorreu. Também podemos remover a coluna "hash".

Além de remover as colunas mencionadas, podemos criar uma nova coluna de nome "id", que contenha um índice referente a cada linha da tabela. Este procedimento pode ser útil caso venhamos a dividir esta tabela em tabelas menores no futuro para criar uma base de dados com tabelas relacionais.

In [ ]:
df_180.drop(['hash', 'uf'], axis=1, inplace=True)

In [ ]:
# Criação de índices com padrão "id_xxxxxx"

str_df = str()
list_str_df = []

for i in range(0, len(df_180)):
    
    str_df = 'id_' + str(i)
    list_str_df.append(str_df)
    
df_180['id'] = list_str_df

Nosso DataFrame após as primeiras simplificações ficou com a forma:

In [ ]:
df_180.shape

As colunas e o tipo dos dados nelas contidos são:

In [ ]:
df_180.dtypes

Notamos então que todas as colunas são do tipo "object". Com o objetivo de manter a consistência em nosso DataFrame, passaremos todas as entradas do tipo string para letras maiúsculas:

In [ ]:
df_180 = df_180.applymap(lambda x: x.upper() if type(x) == str else x)

Observemos novamente as cinco primeiras entradas de nosso DataFrame:

In [ ]:
df_180.head()

Ao investigar brevemente os outputs, podemos notar que as colunas "violacoes", "motivacoes" e "agravantes" parecem conter strings de formatos não intuitivos e com informações redundantes, criando uma estrutura complexa de entradas, como em:

In [ ]:
df_180['motivacoes'].head().value_counts()

Nestes exemplos, as entradas "Motivação.xxxx;Motivação.xxxx;..." não precisariam da palavra "Motivação" escrita tantas vezes. Sabemos que o que vem escrito após o ponto corresponderá a uma motivação do crime, dado o nome da coluna. Também seria mais interessante quebrar esta string longa em uma lista de strings.

Mas antes de procedermos, podemos notar no output produzido após `df.head()` que algumas entradas em "motivacoes" e "agravantes" são do tipo `NaN`. Para garantir a integridade de nossa análise, podemos converter todas as entradas de nossa tabela em strings. Podemos deixar as entradas `NaN` com formato similar ao restante das entradas sem informações na tabela (onde consta "N/D"). Desta forma, a manipulação das strings que não gerará erros:

In [ ]:
df_180.replace(np.nan, 'N/D', inplace=True) # para organizar motivacoes, agravantes e violacoes

Agora, finalmente, podemos aplicar a função criada nos passos iniciais deste projeto. A função quebrará as strings presentes nas colunas de "violacoes", "motivacoes" e "agravantes" em sets de strings menores. Inicialmente aplicada na coluna "violacoes", a função retornará uma variável, aqui chamada de "todas_violacoes", correspondente a um set com todos os tipos de violações presentes na planilha. 

In [ ]:
todas_violacoes = CleanRowData('violacoes')

Investiguemos todos os tipos de violação contidos no set anterior:

In [ ]:
todas_violacoes

Como podemos ver, algumas entradas são redundantes por conta de um erro de digitação (espaço no meio de algumas palavra). Agrupemos estas categorias: 

In [ ]:
palavras = ['PSÍ QUICA', 'CÁ RCERE']
palavras_corr = ['PSÍQUICA', 'CÁRCERE']

for i, item in enumerate(df_180['violacoes']):
    
    for j, jtem in enumerate(df_180['violacoes'][i]):
                
        for k, palavra in enumerate(palavras):
        
            if palavra in jtem:

                df_180['violacoes'][i][j] = re.sub(palavras[k], palavras_corr[k], df_180['violacoes'][i][j])
        


Agora podemos aplicar nossa função de limpeza das strings novamente, apenas para que o set contendo todos os tipos de violações seja atualizado:

In [ ]:
todas_violacoes = CleanRowData('violacoes')
todas_violacoes

Após os procedimentos acima, a coluna de violações ficou da forma:

In [ ]:
df_180['violacoes'].head()

Observemos agora a entrada (lista de strings) de nossa coluna violações que contém o maior número de elementos:

In [ ]:
sorted(max(df_180.violacoes, key=len))

Vemos que vários elementos da lista são, na verdade, repetidos. Para removermos os elementos desnecessários, podemos converter as entradas das colunas em sets.

In [ ]:
df_180['violacoes'] = df_180['violacoes'].apply(set)
# df_180.head()

Agora, o set com o maior número de elementos na coluna de violações possui os seguintes elementos:

In [ ]:
max(df_180.violacoes, key=len)

Agora podemos efetuar os mesmos procedimento, quando estes forem necessários, de forma a limpar as colunas de motivacoes e agravantes:

In [ ]:
todas_motivacoes = CleanRowData('motivacoes')
todas_motivacoes

Podemos ver que não havia nenhuma redundância na coluna 'motivacoes', de forma que podemos transformar as entradas da coluna em sets diretamente.

In [ ]:
df_180['motivacoes'] = df_180['motivacoes'].apply(set)

Limpando agora a coluna dos agravantes:

In [ ]:
todos_agravantes = CleanRowData('agravantes')
todos_agravantes

Observando o set acima, podemos notar que alguns elementos caíram fora de suas categorias mais vastas, o que gerou categorias desnecessárias. Agruparemos os elementos em suas categorias mais vastas e ainda sim específicas:

In [ ]:
words = [['AGRESSOR CÔNJUGE', 'CONVIVENTE', 'ASCENDENTE', 'DESCENDENTE OU PARENTE'],
         ['MOTIVO VIL', 'TORPE', 'INSIDIOSO', 'CRUEL' , 'À TRAIÇÃO', 'OU POR DINHEIRO'],
         ['VÍTIMA IDOSA', 'CRIANÇA', 'DEFICIENTE OU MINORIA ÉTNICA OU SOCIAL']]
        
categories = ['AGRESSOR CÔNJUGE, CONVIVENTE, ASCENDENTE, DESCENDENTE OU PARENTE',
              'MOTIVO VIL, TORPE, INSIDIOSO, CRUEL, À TRAIÇÃO, OU POR DINHEIRO',
              'VÍTIMA IDOSA, CRIANÇA, DEFICIENTE OU MINORIA ÉTNICA OU SOCIAL']

for i, item in enumerate(df_180['agravantes']):
    
    j = 0

    for a, palavras in enumerate(words):
            
        for b, palavra in enumerate(palavras):

            if item.count(palavra) > 1: # duplicatas já são removidas neste passo
                
                item.remove(palavra)

            if palavra in item:
                
                j = 1
    
                item.remove(palavra)

        if (j==1):

            item.append(categories[a])


E agora as categorias ficam da forma:

In [ ]:
todos_agravantes = CleanRowData('agravantes')
todos_agravantes

Transformando as entradas da coluna "agravantes" em sets:

In [ ]:
df_180['agravantes'] = df_180['agravantes'].apply(set)

Nosso DataFrame agora está mais simplificado, e as colunas "motivacoes", "agravantes" e "violacoes" ficaram da forma:

In [ ]:
df_180[['motivacoes', 'violacoes', 'agravantes']].head()

Como sets vazios não correspondem a entradas nulas, substituiremos `set()` por `np.nan`.

In [ ]:
df_180.replace(set(), np.nan, inplace=True) # substituindo sets vazios por np.nan

No próximo passo, verificaremos os dados referentes a outras colunas:

In [ ]:
df_180.columns

In [ ]:
df_180['canal_atendimento'].value_counts()

Simplificando as strings acima:

In [ ]:
str_canal = 'DENÚNCIA - ATENDIMENTO '
str_telefone = 'TELEFÔNICO'

df_180['canal_atendimento'] = df_180['canal_atendimento'].str.replace(str_canal, '')
df_180['canal_atendimento'] = df_180['canal_atendimento'].str.replace(str_telefone, 'TELEFONE')

In [ ]:
df_180['canal_atendimento'].value_counts()

A checagem das outras colunas foi feita como nos casos acima, mas as investigações individuais não constam nesse Jupyter Notebook. Caso seja de seu interesse dar uma olhada nas strings de cada coluna, basta realizar o comando `df['nome_da_coluna'].value_counts()` correspondente à coluna desejada.

<details><summary>Clique aqui para copiar o comando referente à coluna desejada.</summary>

```python
    
# df_180['local_violacao'].value_counts()
# df_180['denunciante'].value_counts()
# df_180['rel_denunciante_vitima'].value_counts()
# df_180['canal_atendimento'].value_counts()
# df_180['pais_vitima'].value_counts()
# df_180['estado'].value_counts()
# df_180['municipio'].value_counts()
# df_180['local_violacao'].value_counts()
# df_180['grupo_vitima'].value_counts()
# df_180['sexo_vitima'].value_counts()
# df_180['idade_vitima'].value_counts()
# df_180['escolaridade_vitima'].value_counts()
# df_180['renda_vitima'].value_counts()
# df_180['raca_vitima'].value_counts()
# df_180['deficiencia_vitima'].value_counts()
# df_180['sexo_suspeito'].value_counts()
# df_180['rel_suspeito_vitima'].value_counts()
# df_180['denuncia_emergencial'].value_counts()
# df_180['nacionalidade_vitima'].value_counts()
# df_180['etnia_vitima'].value_counts()
```

</details>


Neste momento, vamos substituir "N/D" por `np.nan` em todo o DataFrame.

In [ ]:
df_180.replace('N/D', np.nan, inplace=True)

Agora, vamos investigar os dados referentes ao país de origem da vítima. Dado o output abaixo, podemos notar que existem valores `np.nan` na coluna "pais_vitima", mesmo quando existem dados referentes à nacionalidade da vítima.

In [ ]:
for i, item in enumerate(df_180['pais_vitima']):
    
    if pd.isna(item) and (not pd.isna(df_180['nacionalidade_vitima'][i])):
        
        print (i,item, df_180['pais_vitima'][i], df_180['nacionalidade_vitima'][i])
        
        break

Substituiremos `np.nan` por "BRASILEIRO(A)" na coluna 'nacionalidade_vitima' nos casos em que temos entradas correspondentes "BRASIL" na coluna 'pais_vitima':

In [ ]:
df_180['nacionalidade_vitima'] = np.where((pd.isna(df_180['nacionalidade_vitima'])) & (df_180['pais_vitima'] == 'BRASIL'), 'BRASILEIRO(A)', df_180['nacionalidade_vitima'])

Analogamente, atribuiremos o país de origem como "BRASIL" se a nacionalidade da vítima for "BRASILEIRO(A)"

In [ ]:
df_180['pais_vitima'] = np.where((pd.isna(df_180['pais_vitima'])) & (df_180['nacionalidade_vitima'] == 'BRASILEIRO(A)'), 'BRASIL', df_180['pais_vitima'])

Nosso DataFrame está, no momento, da forma:

In [ ]:
df_180.head(7)

Verifiquemos as entradas de 'nacionalidade_vitima' que estão vazias, mas que possuem o país de origem da vítima na coluna 'pais_vitima': 

In [ ]:
for i, item in enumerate(df_180['nacionalidade_vitima']):
    
    if pd.isna(item) and df_180['pais_vitima'][i]!='BRASIL' and not pd.isna(df_180['pais_vitima'][i]):

        print(df_180['pais_vitima'][i],
              df_180['nacionalidade_vitima'][i],
              df_180['estado'][i], 
              df_180['municipio'][i])
        
        break

Da mesma forma, também possuímos linhas em nosso DataFrame que contêm a nacionalidade da vítima, mas que não possuem o país de origem da vítima:

In [ ]:
for i, item in enumerate(df_180['pais_vitima']):
    
    if pd.isna(item) and df_180['nacionalidade_vitima'][i]!='BRASILEIRO(A)' and not pd.isna(df_180['nacionalidade_vitima'][i]):

        print(df_180['pais_vitima'][i],
              df_180['nacionalidade_vitima'][i],
              df_180['estado'][i], 
              df_180['municipio'][i])
        
        break

Notamos então que alguns relatos no Ligue 180 são referentes a vitímas não brasileiras. Não limparemos estes dados, apesar de a maioria deles não possuir informações desejadas para nossa análise, como o estado e o município onde ocorreram as violações. Manteremos estas informações para extrair novos parâmetros de nossa tabela, como o número de pessoas estrangeiras que sofre violência no Brasil e reporta no Canal 180. Apagaremos apenas as linhas que possuem todas as colunas de nacionalidade, pais, estado e municipio vazias:

In [ ]:
df_180 = df_180.dropna(subset=['pais_vitima','nacionalidade_vitima','estado','municipio'], how='all').reset_index(drop=True)

Contando o número de entradas nulas em nosso DataFrame, temos que: 

In [ ]:
df_180.isna().sum()

Baseando-se na quantidade de entradas nulas em nosso DataFrame, podemos inferir que indicadores sociais importantes como etnia da vítima, renda da vítima, raça/cor da vítima, se a vítima é PCD ou não, e até mesmo gênero da vítima não vêm sendo coletados corretamente nas chamadas e contatos com o Ligue 180. Com a ausência de tais informações, pesquisas sobre a violência contra a mulher no Brasil tornam-se mais difíceis. 

Dada a ausência destas informações, não podemos substituir valores de entradas nulas por quaisquer valores, nem mesmo pela moda das entradas. Assumir que a vítima reportada através do Canal possui determinadas características é anti-ético e pode criar tendências enviesadas, talvez corroborando com estereótipos, o que é demasiado problemático. Desta forma, manteremos as entradas nulas da tabela a fim de contabilizar os casos de violência.

Nosso DataFrame ficou então da forma:

In [ ]:
pd.set_option('display.max_columns', None)

df_180.head()

Agora podemos salvar nosso DataFrame com todas as modificações que foram realizadas:

In [ ]:
df_180.to_csv('dataframe_180.csv')

## Bônus - DataFrames adicionais

Para facilitar uma análise futura dos crimes cometidos em cada caso, podemos criar novos DataFrames que possuem os mesmos índices dos casos da nossa tabela `df_180`, mas com apenas com informações referentes às violações, às motivações e aos agravantes. Assim podemos criar uma base de dados com tabelas relacionais, usando como chave a coluna `id`.

Abaixo, criamos a função que vai gerar novos DataFrames:

In [ ]:
def CreateDataFrames(column):
    
    df_180_new = df_180.copy() # cópia de df_180

    df_180_new = df_180_new.filter(items=['id']) # queremos apenas a coluna id
    
    for index, row in df_180.iterrows(): # iterando entre as linhas do dataframe
    
        if pd.notna(row[column]): # função não será aplicada no caso de entradas nulas

            for motivacao in list(row[column]):

                df_180_new.at[index, motivacao] = 1 # criação ou preenchimento de nova coluna,
                                                    # correspondente ao agravante, motivacao ou violacao
    
    df_180_new = df_180_new.fillna(0) # preenchendo as entradas nan com zeros
                
    return (df_180_new)
    

In [ ]:
df_motivacoes = CreateDataFrames('motivacoes')
df_motivacoes.head()

In [ ]:
df_agravantes = CreateDataFrames('agravantes')
df_agravantes.head()

In [ ]:
df_violacoes = CreateDataFrames('violacoes')
df_violacoes.head()

Salvando nossas novas tabelas:

In [ ]:
df_motivacoes.to_csv('dataframe_180_motivacoes.csv')
df_agravantes.to_csv('dataframe_180_agravantes.csv')
df_violacoes.to_csv('dataframe_180_violacoes.csv')

---

Para mais projetos, visite meu repositório do GitHub [aqui](https://github.com/csergilo).